[도전과제] 

 아래는 백화점 고객의 1년 간 구매 데이터이다.
 
(가) 제공 데이터 목록  
① y_train.csv : 고객의 성별 데이터 (학습용), CSV 형식의 파일  
② X_train.csv, X_test.csv : 고객의 상품구매 속성 (학습용 및 평가용), CSV 형식의 파일  
(나) 데이터 형식 및 내용  
① y_train.csv (3,500명 데이터)  
② X_train.csv (3,500명 데이터), X_test.csv (2,482명 데이터)  

고객 3,500명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 
후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객의 성별 예측값(남자일 확률)을 
다음과 같은 형식의 CSV 파일로 생성하시오.(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 
채점)  

<제출형식>
 custid,gender  
 3500,0.267  
 3501,0.578  
 3502,0.885  
 ․ ․ ․ 
 
 <유의사항>  
성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 
알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다

 X_train.csv,y_train.csv 로 train,test 으로 나눠서 평가 0.7정도가 나오면 상당히 높은 편

In [45]:
import pandas as pd
X_df = pd.read_csv('./dataset/X_train.csv',encoding='CP949')
y_df = pd.read_csv('./dataset/y_train.csv',encoding='CP949')

In [46]:
display(X_df.head())
display(y_df.head())

,cust_id,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17
1,1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1
2,2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1
3,3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16
4,4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85


,cust_id,gender
0,0,0
1,1,0
2,2,1
3,3,1
4,4,0


In [27]:
# cust_id는 고유 번호이므로 삭제
X = X_df.drop('cust_id',axis=1)
y = y_df.drop('cust_id',axis=1)

In [28]:
# gender 와의 상관성을 찾기 위하여 gender를 합친 데이터 프레임 생성
Xy = pd.concat([X,y],axis =1)
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,11264000,6860000.0,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,2136000,300000.0,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1639000,NaN,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,4935000,NaN,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,24000000,NaN,보석,본 점,2,1.500000,0.000000,85,0


In [29]:
# 주구매 상품과의 연관성 파악
Xy[['주구매상품','gender']].groupby('주구매상품').mean()

,gender
주구매상품,
가공식품,0.419414
가구,0.500000
건강식품,0.531915
골프,0.536585
구두,0.518519
기타,0.369748
남성 캐주얼,0.763636
남성 트랜디,0.500000
남성정장,0.818182


In [30]:
# 라벨 인코딩으로 주구매상품 분류
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X['주구매상품'] = le.fit_transform(X['주구매상품'])
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,강남점,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,잠실점,2,1.500000,0.000000,1
2,3197000,1639000,NaN,6,관악점,2,2.000000,0.000000,1
3,16077620,4935000,NaN,5,광주점,18,2.444444,0.318182,16
4,29050000,24000000,NaN,15,본 점,2,1.500000,0.000000,85


In [31]:
# 주구매 상품과의 연관성 파악
Xy[['주구매지점','gender']].groupby('주구매지점').mean()

,gender
주구매지점,
강남점,0.255172
관악점,0.413043
광주점,0.403509
노원점,0.311111
대구점,0.714286
대전점,0.357143
동래점,0.346939
미아점,0.434783
본 점,0.402971


In [32]:
# 주구매지점 역시 라벨인코딩으로 변환
X['주구매지점'] = le.fit_transform(X['주구매지점'])
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,6860000.0,5,0,19,3.894737,0.527027,17
1,2136000,2136000,300000.0,21,19,2,1.500000,0.000000,1
2,3197000,1639000,NaN,6,1,2,2.000000,0.000000,1
3,16077620,4935000,NaN,5,2,18,2.444444,0.318182,16
4,29050000,24000000,NaN,15,8,2,1.500000,0.000000,85


In [33]:
# 환불 여부와 성별 상관관계 파악
import numpy as np
Xy.환불금액.fillna(0,inplace = True)
Xy['환불금액'] = np.where(Xy.환불금액 == 0 ,0,1)
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,11264000,1,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,2136000,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1639000,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,4935000,0,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,24000000,0,보석,본 점,2,1.500000,0.000000,85,0


In [34]:
Xy[['환불금액','gender']].groupby('환불금액').mean()

,gender
환불금액,
0,0.447059
1,0.240664


In [35]:
# X에 Xy의 데이터로 대체
X['환불금액']=Xy['환불금액']
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,11264000,1,5,0,19,3.894737,0.527027,17
1,2136000,2136000,1,21,19,2,1.500000,0.000000,1
2,3197000,1639000,0,6,1,2,2.000000,0.000000,1
3,16077620,4935000,0,5,2,18,2.444444,0.318182,16
4,29050000,24000000,0,15,8,2,1.500000,0.000000,85


In [36]:
# 최대 구매액 범주화
X.최대구매액.describe()
def get_catmax(X):
    cat = ''                #여기에 저장됨
    if X < 2.875000e+06 : cat = 1
    elif X < 9.837000e+06: cat = 2
    elif X < 2.296250e+07: cat = 3
    else:
        cat = 4
    return cat
Xy['최대구매액'] = Xy['최대구매액'].apply(lambda X : get_catmax(X))
Xy.head()


,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,68282840,3,1,기타,강남점,19,3.894737,0.527027,17,0
1,2136000,1,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,3197000,1,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,16077620,2,0,기타,광주점,18,2.444444,0.318182,16,1
4,29050000,4,0,보석,본 점,2,1.500000,0.000000,85,0


In [37]:
# 나누어진 범주로 성별과의 연관성 파악
Xy[['최대구매액','gender']].groupby('최대구매액').mean()

,gender
최대구매액,
1,0.522883
2,0.372146
3,0.342857
4,0.266286


In [38]:
# X에도 범주 적용
X['최대구매액'] = X['최대구매액'].apply(lambda X : get_catmax(X))
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,68282840,3,1,5,0,19,3.894737,0.527027,17
1,2136000,1,1,21,19,2,1.500000,0.000000,1
2,3197000,1,0,6,1,2,2.000000,0.000000,1
3,16077620,2,0,5,2,18,2.444444,0.318182,16
4,29050000,4,0,15,8,2,1.500000,0.000000,85


In [39]:
Xy.총구매액.describe()

count    3.500000e+03
mean     9.191925e+07
std      1.635065e+08
min     -5.242152e+07
25%      4.747050e+06
50%      2.822270e+07
75%      1.065079e+08
max      2.323180e+09
Name: 총구매액, dtype: float64

In [40]:
# 총 구매액 범주화
def get_catall(X):
    cat = ''                #여기에 저장됨
    if X < 4.747050e+06 : cat = 1
    elif X < 2.822270e+07 : cat = 2
    elif X < 1.065079e+08 : cat = 3
    else:
        cat = 4
    return cat
Xy['총구매액'] = Xy['총구매액'].apply(lambda X : get_catall(X))
Xy.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기,gender
0,3,3,1,기타,강남점,19,3.894737,0.527027,17,0
1,1,1,1,스포츠,잠실점,2,1.500000,0.000000,1,0
2,1,1,0,남성 캐주얼,관악점,2,2.000000,0.000000,1,1
3,2,2,0,기타,광주점,18,2.444444,0.318182,16,1
4,3,4,0,보석,본 점,2,1.500000,0.000000,85,0


In [41]:
# 나누어진 범주로 성별과의 연관성 파악
Xy[['총구매액','gender']].groupby('총구매액').mean()

,gender
총구매액,
1,0.507429
2,0.427429
3,0.336000
4,0.233143


In [19]:
# X에도 범주 적용
X['총구매액'] = X['총구매액'].apply(lambda X : get_catall(X))
X.head()

,총구매액,최대구매액,환불금액,주구매상품,주구매지점,내점일수,내점당구매건수,주말방문비율,구매주기
0,3,3,1,5,0,19,3.894737,0.527027,17
1,1,1,1,21,19,2,1.500000,0.000000,1
2,1,1,0,6,1,2,2.000000,0.000000,1
3,2,2,0,5,2,18,2.444444,0.318182,16
4,3,4,0,15,8,2,1.500000,0.000000,85


In [59]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,f1_score

def score_cal(y_test,y_pred=None,y_pred_proba=None):
    acc = accuracy_score(y_test,y_pred)
    pre = precision_score(y_test,y_pred)
    rec = recall_score(y_test,y_pred)
    f1 = f1_score(y_test,y_pred)    
    roc = roc_auc_score(y_test,y_pred_proba)
    print(f'accuracy : {acc}, precision : {pre}, recall : {rec}, f1_score : {f1}, roc_auc : {roc}')

X_train ,X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=11)
lr = LogisticRegression()
rf = RandomForestClassifier()
dt = DecisionTreeClassifier()

lr.fit(X_train,y_train)
rf.fit(X_train,y_train)
dt.fit(X_train,y_train)

lr_pred = lr.predict(X_test)
lr_pred_proba = lr.predict_proba(X_test)[:,1]
rf_pred = rf.predict(X_test)
rf_pred_proba = rf.predict_proba(X_test)[:,1]
dt_pred = dt.predict(X_test)
dt_pred_proba = dt.predict_proba(X_test)[:,1]

C:\Users\admin\anaconda3\envs\cakd5\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\admin\AppData\Local\Temp/ipykernel_9856/1632665009.py:21: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)


In [62]:
score_cal(y_test,lr_pred,lr_pred_proba)

accuracy : 0.6257142857142857, precision : 0.0, recall : 0.0, f1_score : 0.0, roc_auc : 0.5975789629862366


In [61]:
score_cal(y_test,rf_pred,rf_pred_proba)

accuracy : 0.6228571428571429, precision : 0.49044585987261147, recall : 0.2950191570881226, f1_score : 0.368421052631579, roc_auc : 0.6149512563384215


In [60]:
score_cal(y_test,dt_pred,dt_pred_proba)

accuracy : 0.57, precision : 0.421259842519685, recall : 0.4099616858237548, f1_score : 0.4155339805825243, roc_auc : 0.5375548748025379
